In [1]:
import os
import mat73
import pandas as pd
from tqdm import tqdm
import numpy as np


In [2]:
def normalization(fluorescence, optical_props):
    f = (fluorescence - np.mean(fluorescence, axis=(1,2,3), keepdims=True)) / \
                   (np.std(fluorescence, axis=(1,2,3), keepdims=True) + 1e-6)
    mu_a = optical_props[..., 0]
    mu_s = optical_props[..., 1]

    mu_a_mean = np.mean(mu_a, axis=(1,2), keepdims=True)
    mu_a_std = np.std(mu_a, axis=(1,2), keepdims=True)
    mu_a_norm = (mu_a - mu_a_mean) / (mu_a_std + 1e-6)

    mu_s_mean = np.mean(mu_s, axis=(1,2), keepdims=True)
    mu_s_std = np.std(mu_s, axis=(1,2), keepdims=True)
    mu_s_norm = (mu_s - mu_s_mean) / (mu_s_std + 1e-6)
    return f, mu_a_norm, mu_s_norm


In [3]:
def scale_data(data_dict, params):
    scaled_data_dict = {}
    for key, items in data_dict.items():
        scaled_data_dict[key] = items * params[key]
    return scaled_data_dict

In [4]:
def read_data(path, params):
    data = mat73.loadmat(path)

    fluorescence = data['F']
    optical_props = data['OP']
    reflectance = data['RE']
    depth = data['DF']
    concentration = data['QF']

    norm_fluorescence, norm_mu_a, norm_mu_s = normalization(fluorescence, optical_props)
    scaled_data_dict = scale_data({
        'fluorescence': norm_fluorescence,
        'mu_a': norm_mu_a,
        'mu_s': norm_mu_s,
        'reflectance': reflectance,
        'depth': depth,
        'concentration': concentration
    }, params)

    scaled_data_dict['fluorescence'] = scaled_data_dict['fluorescence'].transpose(0, 3, 1, 2)
    op = np.stack([scaled_data_dict['mu_a'], scaled_data_dict['mu_s']], axis=1)
    scaled_data_dict['op'] = op
    scaled_data_dict['reflectance'] = scaled_data_dict['reflectance'].transpose(0, 3, 1, 2)
    scaled_data_dict['depth'] = scaled_data_dict['depth']
    scaled_data_dict['concentration'] = scaled_data_dict['concentration']
    del scaled_data_dict['mu_a']
    del scaled_data_dict['mu_s']

    return scaled_data_dict


In [5]:
scale_params = {
    'fluorescence': 10e4,
    'mu_a': 10,
    'mu_s': 1,
    'reflectance': 1,
    'depth': 1,
    'concentration': 1
}
path = '/home/victorhuang/projects/gtx/data/phantom_data.mat'
results = read_data(path, scale_params)